# RISC-V Qemu

Since there are no available RISCV64 CPUs just yet, we need to use Qemu with the latest RISC-V patches to run our distributions.

Following this Notebook you can have a development or testing machine ready to use in very little time at all.

Note: This guide is still a work in progress.
TODO:
- Add comments about what the commands do
- Add info about how to create slack.img

This tutorial is based on [Riscv Qemu at Github](https://github.com/riscv/riscv-qemu) and the blog about [Fedora stage4](https://rwmj.wordpress.com/2018/02/15/fedora-risc-v-runnable-stage-4-disk-images/).

While I wait to get a RiscV64 chip to play with compiler options and performance, I am using some beafy servers provided by [Greencore Solutions](https://www.greencore.co.cr/) that run Ubuntu. If you need help with any other distribution, please let me know and I'll add it to the tutorial.

Let's start by installing basic packages.

In [ ]:
cd ~
sudo apt-get install -y git flex bison lftp
sudo apt-get build-dep -y qemu

We now download the code, create a build directory and configure, make and make install.

In [ ]:
git clone https://github.com/riscv/riscv-qemu.git
cd riscv-qemu
mkdir build
cd build
../configure
make -j6
sudo make install

Download the fedora stage4, this will only be necesary while the Slackware RISCV stage4 is ready.

In [ ]:
cd ~
mkdir fedora-stage4
cd fedora-stage4
lftp -e "mirror -c;exit" https://fedorapeople.org/groups/risc-v/disk-images/

Now, uncompress the stage4 to make it available to Qemu, and also create a secondary device for development.

In [ ]:
xz -dk stage4-disk.img.xz
fallocate -l 40G slack.img
mkfs.ext4 slack.img

**From a terminal**:
```
sudo qemu-system-riscv64 \
  -nographic -machine virt -m 6G -smp 6 \
  -kernel bbl \
  -append "console=ttyS0 ro root=/dev/vda init=/usr/sbin/init" \
  -device virtio-blk-device,drive=hd0 \
  -drive file=stage4-disk.img,format=raw,id=hd0 \
  -device virtio-net-device,netdev=usernet \
  -netdev tap,id=usernet \
  -device virtio-blk-device,drive=hd1 \
  -drive file=slack.img,format=raw,id=hd1
```

After the stage4 has booted, you can now connect to it via ssh.

I would recommend adding this basic tools to Fedora:
```
dnf install -y vim screen git bison byacc rsync libtool ncurses-devel gettext attr python2 ccache \
  libuuid-devel expat-devel libcap-ng-devel zlib-devel device-mapper-devel popt-devel libblkid-devel \
  libgcrypt-devel python2-devel
```

For Slackware development, inside the Fedora stage4:
```
cd /
tar xJvf /mnt/slackware-current/slackware/a/pkgtools-15.0-noarch-6.txz
export PATH="/usr/lib64/ccache/:$PATH:/sbin"
```